# **In-processing technique: Exponentiated Gradient Reduction for Statistical Parity**

We will illustrate how to mitigate bias in a binary classification context. Let's imagine that a company is looking to hire a new employee. They use a machine learning algorithm to select the top candidates. The candidates are assigned either 0 if they are not selected or 1 if they are. 

In this notebook, we address bias by intervening at the in-processing stage. The Gradient Reduction approach (Agarwal et al 2018) makes use of constrained optimisation to reduce binary classification to a series of cost-sensitive, weighted classification problems. The optimal solution is then an equilibrium between two min max expressions. The steps we will take are outlined below.

1. First, we will calculate fairness metrics for a base classifier
2. We will then apply the Reductions method to train a predictive model and observe the results.

## **Install Libraries and load data**

In [ ]:
# install AIF360
!pip install aif360
!pip install fairlearn==0.7.0

import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from fairlearn.metrics import MetricFrame
from fairlearn.metrics import demographic_parity_difference
from fairlearn.metrics import equalized_odds_difference
from fairlearn.metrics import false_negative_rate_difference
from fairlearn.reductions import DemographicParity
from fairlearn.reductions import EqualizedOdds
from fairlearn.reductions import ExponentiatedGradient

from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import RidgeClassifier


Please download the data from the following link: https://hai-data.s3.eu-west-2.amazonaws.com/roadmaps/hiring.pkl. If running in Colab, please upload the data to the local folder. Otherwise, place the data in the same folder as the notebook. Load the data into a dataframe using pickle.load and print the dataframe to inspect it.


In [ ]:
# load data
with open('hiring.pkl', 'rb') as handle:
    df = pickle.load(handle)


In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,497,498,499,Gender_Female,Gender_Male,Ethnicity_Asian,Ethnicity_Black,Ethnicity_Hispanic,Ethnicity_White,Action
0,28.021737,4.351153,2.453895,1.637143,-1.746628,-0.483463,0.034170,1.399225,-0.795440,0.417474,...,0.258914,-0.050558,0.014513,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,29.603342,-3.407193,0.771800,-2.957411,0.599226,-2.805277,0.329414,-2.055339,-1.194446,-0.633159,...,0.442939,-0.054423,0.026959,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,26.504283,0.642464,2.522944,-2.197094,2.270646,-0.472510,0.532815,-0.266449,-0.131638,1.038315,...,0.017111,-0.012309,0.264572,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,25.012088,0.895121,-2.092517,3.687830,0.539642,1.988930,1.121646,2.255337,-0.128801,1.148379,...,-0.149901,-0.217130,0.004403,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,27.358934,-2.332423,0.154999,-2.623793,1.682456,1.262280,-1.685565,0.489319,-0.043471,-0.372265,...,0.033429,-0.199198,0.229629,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31765,28.366808,3.697922,3.563571,-2.150857,-0.434302,0.678316,-2.203074,-0.244846,-0.568165,-0.760018,...,0.131910,-0.143816,-0.313901,0.0,1.0,0.0,0.0,1.0,0.0,1.0
31766,27.366891,-1.487017,-3.040194,-0.231647,-0.557333,-1.919395,1.733025,-0.523544,-1.182572,0.001023,...,0.114989,-0.511371,0.065787,1.0,0.0,0.0,0.0,1.0,0.0,1.0
31768,26.421217,2.768473,4.175005,-2.132873,-1.590674,2.238180,1.423327,-1.875555,0.495852,-0.262379,...,0.201736,-0.289361,-0.148308,0.0,1.0,0.0,0.0,0.0,1.0,0.0
31769,27.286430,5.486510,-2.384405,-0.535356,-1.621977,1.019637,1.298022,-0.809889,-1.387841,-0.497270,...,-0.252254,-0.073340,0.556241,1.0,0.0,0.0,0.0,1.0,0.0,1.0


## **Run a baseline predictive model without applying a fairness technique**
First we will build a standard Ridge Classifier and observe some baseline results, using the original data and without any bias mitigation technique.

We train a Ridge Classifier with 10 fold stratified cross validation, compute performance metrics (Accuracy, Precision, Recall and F1 Score) and bias metrics (Average Odds Difference, False Negative Rate Difference, and Statistical Parity Difference). 

In this example we will assign 'Ethnicity_White' as our privileged group. All other ethnicities are assigned as unprivileged.


In [ ]:
# Instantiate the classifier

model = RidgeClassifier()

# instantiate the cross-validation scheme
mv = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)

# setup the performance metrics to be computed
perf_metrics = {"Accuracy": metrics.accuracy_score, 
                "Precision": metrics.precision_score, 
                "Recall": metrics.recall_score, 
                "F1-Score": metrics.f1_score, 
                }


In [ ]:
# Train a baseline ridge regression classifier on the dataset before applying the Reductions method
k, i = True, 1

# instantiating X
df.columns = df.columns.astype(str)
X = df.drop(labels=df.columns[-1], axis=1)

# instantiating the target variable
y = df[df.columns[-1]]

# 10 fold CV
for (train, test) in mv.split(X, y):

    # fit model
    X_train = X.iloc[train].copy()
    y_train = y.iloc[train].copy()
    model = model.fit(X_train, y_train)
    
    # get predictions in the test set
    ypred_class = model.predict(X.iloc[test])

    # compute performance metrics
    metrics = []
    X_test = X.iloc[test].copy()
    y_test = y.iloc[test].copy()
    white = X_test['Ethnicity_White']
    for pf in perf_metrics.keys():
            metrics += [[pf, perf_metrics[pf](y.iloc[test].values.ravel(), ypred_class)]]
    spd = demographic_parity_difference(y_test, ypred_class, sensitive_features=white)
    eod = equalize_odds_difference(y_test, ypred_class, sensitive_features=white)
    fnrd = false_negative_rate_difference(y_test, ypred_class, sensitive_features=white)
    
    # Compute fairness metrics
    metrics += [['Statistical Parity Difference', spd]]
    metrics += [['Average Odds Difference', eod]]
    metrics += [['False Negative Rate Difference', fnrd]]

    # concatenate results
    df_m = pd.DataFrame(metrics, columns=["Metric", "Value"])
    df_m["Fold"] = i
    i += 1
    if k:
        df_metrics_orig = df_m.copy()
        k=0
    else:
        df_metrics_orig = pd.concat([df_metrics_orig, df_m.copy()], axis=0, ignore_index=True)

In [ ]:
# Display metrics

metrics_table_orig = df_metrics_orig.pivot_table(index="Metric", values="Value", aggfunc=["mean", "std"])
metrics_table_orig

,mean,std
,Value,Value
Metric,,
Accuracy,0.712501,0.007711
Equalized Odds Difference,0.069589,0.025648
F1-Score,0.589124,0.012598
False Negative Rate Difference,0.065355,0.031358
Precision,0.653016,0.012121
Recall,0.536745,0.015263
Statistical Parity Difference,0.063249,0.012323


## **Use the Reductions approach to target Statistical Parity**



We now extend our Ridge Classifier training routine to apply the Exponentiated Gradient in-processing technique. Please notice that this part of the code may take a while to run. 

In [ ]:
# Train a ridge regression classifier on the dataset before reweighing
model = RidgeClassifier()
k, i = True, 1

# instantiating X
X = df.drop(labels=df.columns[-1], axis=1)

# instantiating the target variable
y = df[df.columns[-1]]

# 10 fold cv
for (train, test) in mv.split(X, y):

    # fit reductions in-processor
    X_train = X.iloc[train].copy()
    y_train = y.iloc[train].copy()
    white_train = X_train['Ethnicity_White']

    # Set up the in-processor, we target Statstial/Demographic parity here
    reduce = ExponentiatedGradient(model, DemographicParity())
    reduce.fit(X_train, y_train, sensitive_features=white_train)

    # Get predictions in the test set
    X_test = X.iloc[test].copy()
    ypred_class = reduce.predict(X_test)

    # compute performance metrics
    metrics = []
    y_test = y.iloc[test].copy()
    white = X_test['Ethnicity_White']
    for pf in perf_metrics.keys():
            metrics += [[pf, perf_metrics[pf](y.iloc[test].values.ravel(), ypred_class)]]
    spd = demographic_parity_difference(y_test, ypred_class, sensitive_features=white)
    eod = equalize_odds_difference(y_test, ypred_class, sensitive_features=white)
    fnrd = false_negative_rate_difference(y_test, ypred_class, sensitive_features=white)
    
    # Compute fairness metrics
    metrics += [['Statistical Parity Difference', spd]]
    metrics += [['Average Odds Difference', eod]]
    metrics += [['False Negative Rate Difference', fnrd]]

    # concatenate results
    df_m = pd.DataFrame(metrics, columns=["Metric", "Value"])
    df_m["Fold"] = i
    i += 1
    if k:
        df_metrics = df_m.copy()
        k=0
    else:
        df_metrics = pd.concat([df_metrics, df_m.copy()], axis=0, ignore_index=True)

We now compute performance metrics (Accuracy, Precision, Recall and F1 Score) and fairness metrics (Average Odds Difference, False Negative Rate Difference, Statistical Parity Difference)

In [ ]:
# Display metrics

metrics_table_rw = df_metrics.pivot_table(index="Metric", values="Value", aggfunc=["mean", "std"])
metrics_table_rw

,mean,std
,Value,Value
Metric,,
Accuracy,0.713474,0.008660
Equalized Odds Difference,0.032261,0.021809
F1-Score,0.591006,0.013589
False Negative Rate Difference,0.031055,0.023390
Precision,0.654142,0.013514
Recall,0.539088,0.015507
Statistical Parity Difference,0.015090,0.010106


We can see that the Statistical Parity difference is closer to its target value 0. 